# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 2.205296596065241
 1.8551229792583508
 0.8533886927959882

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [2]:
Alu = lu(A)

LU{Float64, Matrix{Float64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.513615  1.0       0.0
 0.295738  0.271404  1.0
U factor:
3×3 Matrix{Float64}:
 0.83195  0.576718   0.446455
 0.0      0.687695   0.564783
 0.0      0.0       -0.0351695

In [3]:
typeof(Alu)

LU{Float64, Matrix{Float64}}

The different parts of the factorization can be extracted by accessing their special properties

In [4]:
Alu.P

3×3 Matrix{Float64}:
 0.0  1.0  0.0
 1.0  0.0  0.0
 0.0  0.0  1.0

In [5]:
Alu.L

3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.513615  1.0       0.0
 0.295738  0.271404  1.0

In [6]:
Alu.U

3×3 Matrix{Float64}:
 0.83195  0.576718   0.446455
 0.0      0.687695   0.564783
 0.0      0.0       -0.0351695

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [7]:
A\b

3-element Vector{Float64}:
 0.9999999999999999
 1.0000000000000002
 1.0

In [8]:
Alu\b

3-element Vector{Float64}:
 0.9999999999999999
 1.0000000000000002
 1.0

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [9]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [10]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.441843   0.858892  -0.258997
 -0.860261  -0.487536  -0.149194
 -0.254412   0.156884   0.954286
R factor:
3×3 Matrix{Float64}:
 -0.96709  -1.02174   -0.798571
  0.0       0.619937   0.503618
  0.0       0.0       -0.0335617

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [11]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.441843   0.858892  -0.258997
 -0.860261  -0.487536  -0.149194
 -0.254412   0.156884   0.954286

In [12]:
Aqr.R

3×3 Matrix{Float64}:
 -0.96709  -1.02174   -0.798571
  0.0       0.619937   0.503618
  0.0       0.0       -0.0335617

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [13]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.871021215213241
 -0.02678184303514708
  3.406139917482376
vectors:
3×3 Matrix{Float64}:
  0.768088  -0.0906084  -0.633901
 -0.596429  -0.461515   -0.656716
 -0.233051   0.882493   -0.408526

The values and the vectors can be extracted from the Eigen type by special indexing

In [14]:
AsymEig.values

3-element Vector{Float64}:
 -0.871021215213241
 -0.02678184303514708
  3.406139917482376

In [15]:
AsymEig.vectors

3×3 Matrix{Float64}:
  0.768088  -0.0906084  -0.633901
 -0.596429  -0.461515   -0.656716
 -0.233051   0.882493   -0.408526

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [16]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
  1.0           5.32907e-15  4.88498e-15
  2.84217e-14   1.0          2.22045e-14
 -5.68434e-14  -6.03961e-14  1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [17]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [18]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [19]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

2.9631367153816814

In [20]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [21]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [22]:
@time eigvals(Asym);

  0.258496 seconds (134.03 k allocations: 15.456 MiB, 6.10% gc time, 38.03% compilation time)


In [23]:
@time eigvals(Asym_noisy);

  1.050047 seconds (13 allocations: 7.920 MiB)


In [24]:
@time eigvals(Asym_explicit);

  0.149938 seconds (5.88 k allocations: 8.358 MiB, 2.11% compilation time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [25]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.669902 seconds (823.18 k allocations: 229.291 MiB, 12.15% gc time, 26.00% compilation time)


6.7249639294089665

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [26]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [27]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 1//10  3//5   2//5
 1//5   1//10  1//5
 1//5   1//5   3//10

In [28]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
 11//10
  1//2
  7//10

In [29]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1//1
 1//1
 1//1

In [30]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}}
L factor:
3×3 Matrix{Rational{BigInt}}:
 1//1  0//1   0//1
 1//2  1//1   0//1
 1//1  2//11  1//1
U factor:
3×3 Matrix{Rational{BigInt}}:
 1//5   1//10  1//5
 0//1  11//20  3//10
 0//1   0//1   1//22

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [ ]:
using LinearAlgebra

In [ ]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [ ]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [ ]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]